# Data Pre-processing of the AQI Dataset

## Data Preprocessing
**Objectives:**
- Import the dataset and parse dates.
- Handle missing data and outliers.
  

In [116]:
import pandas as pd
import numpy
pd.options.mode.chained_assignment = None

In [70]:
openaq = pd.read_csv('data/openaq.csv', low_memory=False, sep=";").copy()
print(openaq.shape)

(61177, 10)


In [60]:
openaq.head(5) # view the data

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
0,ZA,Alfred Nzo,Luka,"-25.5077634717075, 27.1838594119125",PM10,South Africa,µg/m³,20.0000,2023-04-25T00:00:00+01:00,South Africa
1,IN,NaN,"ICRISAT Patancheru, Hyderabad - TSPCB","17.5184, 78.278777",SO2,caaqm,µg/m³,8.9000,2022-10-31T01:45:00+00:00,India
2,IT,via Cadello CAGLIARI - Cagliari (CA),CENCA1,"39.23583300015611, 9.115000000173477",SO2,eea,µg/m³,0.5123,2025-01-11T21:00:00+00:00,Italy
3,JP,NaN,南アルプス市鏡中條１６４２－２,"35.601667, 138.498611",NO2,japan-soramame,ppm,0.0020,2024-09-23T08:00:00+01:00,Japan
4,KR,NaN,대불,"34.757839999999995, 126.452808",PM10,korea-air,µg/m³,24.0000,2025-01-28T09:00:00+00:00,"Korea, Republic of"


## NaN value handling
First I will consider how to handle missing values. I know there are 61,177 rows of data with 10 columns in the raw data set (as seen when reading the file and printing the shape), so I want to look at how many missing values each column has to get a better idea of how to treat them.

In [40]:
openaq.isna().sum()

Country Code         0
City             29146
Location             2
Coordinates         70
Pollutant            0
Source Name          0
Unit                 0
Value                0
Last Updated         0
Country Label      127
dtype: int64

Here I can see that about 48% of the data is missing the `City` value. Other columns that are missing data have very few missing data counts and so we can just remove these rows, but we will handle the City column differently. We will also now introduce df as the name of our edited dataframe.

In [118]:
# Dropping the rows that have missing data except for missing data in City column
df = openaq.dropna(subset=[col for col in openaq.columns if col != 'City'])

In [104]:
df.isna().sum()

Country Code         0
City             29119
Location             0
Coordinates          0
Pollutant            0
Source Name          0
Unit                 0
Value                0
Last Updated         0
Country Label        0
dtype: int64

## Parsing dates
We will now also parse the `Last Updated` column. From earlier we can see that it is in a ISO 8601 format, we will convert to YYYY-mm-dd HH:MM:ss format

In [112]:
df['Last Updated'] = pd.to_datetime(df['Last Updated'], utc=True)
df['Last Updated'] = df['Last Updated'].dt.strftime('%Y-%m-%d %H:%M:%S')

/var/folders/mw/jb7q0lcd4ys8np0l14s9ksq00000gn/T/ipykernel_49125/3440053547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Last Updated'] = pd.to_datetime(df['Last Updated'], utc=True)
/var/folders/mw/jb7q0lcd4ys8np0l14s9ksq00000gn/T/ipykernel_49125/3440053547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Last Updated'] = df['Last Updated'].dt.strftime('%Y-%m-%d %H:%M:%S')


,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
0,ZA,Alfred Nzo,Luka,"-25.5077634717075, 27.1838594119125",PM10,South Africa,µg/m³,20.000000,2023-04-24 23:00:00,South Africa
1,IN,NaN,"ICRISAT Patancheru, Hyderabad - TSPCB","17.5184, 78.278777",SO2,caaqm,µg/m³,8.900000,2022-10-31 01:45:00,India
2,IT,via Cadello CAGLIARI - Cagliari (CA),CENCA1,"39.23583300015611, 9.115000000173477",SO2,eea,µg/m³,0.512300,2025-01-11 21:00:00,Italy
3,JP,NaN,南アルプス市鏡中條１６４２－２,"35.601667, 138.498611",NO2,japan-soramame,ppm,0.002000,2024-09-23 07:00:00,Japan
4,KR,NaN,대불,"34.757839999999995, 126.452808",PM10,korea-air,µg/m³,24.000000,2025-01-28 09:00:00,"Korea, Republic of"
...,...,...,...,...,...,...,...,...,...,...
61172,ZA,Garden Route,George-NAQI,"-33.981111, 22.473111",NOX,South Africa,ppm,0.012899,2023-03-29 06:00:00,South Africa
61173,RO,PLOIESTI,PH-5,"44.92203299990539, 26.033989000020796",PM10,eea,µg/m³,-1.000000,2024-09-07 22:00:00,Romania
61174,RO,CRAIOVA,DJ-2,"44.319138999966434, 23.796707999854295",PM10,eea,µg/m³,19.373112,2024-09-07 22:00:00,Romania
61175,RO,Tulcea,RO0200A,"45.1804846878482, 28.7874115948942",SO2,EEA Romania,µg/m³,6.942366,2023-08-05 13:00:00,Romania
